<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 300, "amount": -790}, {"transaction-id": 331, "amount": -510}, {"transaction-id": 417, "amount": -658}, {"transaction-id": 654, "amount": -665}, {"transaction-id": 838, "amount": -682}, {"transaction-id": 1041, "amount": -674}, {"transaction-id": 1660, "amount": -739}, {"transaction-id": 1870, "amount": -740}, {"transaction-id": 2025, "amount": -697}, {"transaction-id": 2211, "amount": -729}, {"transaction-id": 2523, "amount": -671}, {"transaction-id": 2628, "amount": -726}, {"transaction-id": 2642, "amount": -768}, {"transaction-id": 2948, "amount": -681}, {"transaction-id": 3388, "amount": -862}, {"transaction-id": 3604, "amount": -920}, {"transaction-id": 3732, "amount": -736}, {"transaction-id": 3981, "amount": -877}, {"transaction-id": 4048, "amount": -917}, {"transaction-id": 4173, "amount": -648}, {"transaction-id": 4428, "amount": -790}, {"transaction-id": 5061, "amount": -886}, {"transaction-id": 5344, "amount"

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 300, "amount": -790}, {"transaction-id": 331, "amount": -510}, {"transaction-id": 417, "amount": -658}, {"transaction-id": 654, "amount": -665}, {"transaction-id": 838, "amount": -682}, {"transaction-id": 1041, "amount": -674}, {"transaction-id": 1660, "amount": -739}, {"transaction-id": 1870, "amount": -740}, {"transaction-id": 2025, "amount": -697}, {"transaction-id": 2211, "amount": -729}, {"transaction-id": 2523, "amount": -671}, {"transaction-id": 2628, "amount": -726}, {"transaction-id": 2642, "amount": -768}, {"transaction-id": 2948, "amount": -681}, {"transaction-id": 3388, "amount": -862}, {"transaction-id": 3604, "amount": -920}, {"transaction-id": 3732, "amount": -736}, {"transaction-id": 3981, "amount": -877}, {"transaction-id": 4048, "amount": -917}, {"transaction-id": 4173, "amount": -648}, {"transaction-id": 4428, "amount": -790}, {"transaction-id": 5061, "amount": -886}, {"transaction-id": 5344, "amount"

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Charlie',
  'transactions': [{'transaction-id': 300, 'amount': -790},
   {'transaction-id': 331, 'amount': -510},
   {'transaction-id': 417, 'amount': -658},
   {'transaction-id': 654, 'amount': -665},
   {'transaction-id': 838, 'amount': -682},
   {'transaction-id': 1041, 'amount': -674},
   {'transaction-id': 1660, 'amount': -739},
   {'transaction-id': 1870, 'amount': -740},
   {'transaction-id': 2025, 'amount': -697},
   {'transaction-id': 2211, 'amount': -729},
   {'transaction-id': 2523, 'amount': -671},
   {'transaction-id': 2628, 'amount': -726},
   {'transaction-id': 2642, 'amount': -768},
   {'transaction-id': 2948, 'amount': -681},
   {'transaction-id': 3388, 'amount': -862},
   {'transaction-id': 3604, 'amount': -920},
   {'transaction-id': 3732, 'amount': -736},
   {'transaction-id': 3981, 'amount': -877},
   {'transaction-id': 4048, 'amount': -917},
   {'transaction-id': 4173, 'amount': -648},
   {'transaction-id': 4428, 'amount': -790},
   {'transac

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 61,
  'name': 'Alice',
  'transactions': [{'transaction-id': 232, 'amount': -2199},
   {'transaction-id': 619, 'amount': -2681},
   {'transaction-id': 696, 'amount': -2537},
   {'transaction-id': 866, 'amount': -2621},
   {'transaction-id': 903, 'amount': -2042},
   {'transaction-id': 1031, 'amount': -2784},
   {'transaction-id': 1086, 'amount': -3101},
   {'transaction-id': 2502, 'amount': -2485},
   {'transaction-id': 2866, 'amount': -2532},
   {'transaction-id': 3225, 'amount': -2478},
   {'transaction-id': 3705, 'amount': -2368},
   {'transaction-id': 3725, 'amount': -2753},
   {'transaction-id': 3813, 'amount': -2698},
   {'transaction-id': 3817, 'amount': -2244},
   {'transaction-id': 4476, 'amount': -3024},
   {'transaction-id': 4561, 'amount': -2474},
   {'transaction-id': 4707, 'amount': -2091},
   {'transaction-id': 4847, 'amount': -2664},
   {'transaction-id': 4946, 'amount': -2426},
   {'transaction-id': 5356, 'amount': -2701},
   {'transaction-id': 5582, 'amount': 

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 30},
 {'name': 'Alice', 'count': 46},
 {'name': 'Alice', 'count': 30},
 {'name': 'Alice', 'count': 3},
 {'name': 'Alice', 'count': 131})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(30, 46, 30, 3, 131)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

42.391117478510026

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 232, 'amount': -2199},
  {'transaction-id': 619, 'amount': -2681},
  {'transaction-id': 696, 'amount': -2537},
  {'transaction-id': 866, 'amount': -2621},
  {'transaction-id': 903, 'amount': -2042},
  {'transaction-id': 1031, 'amount': -2784},
  {'transaction-id': 1086, 'amount': -3101},
  {'transaction-id': 2502, 'amount': -2485},
  {'transaction-id': 2866, 'amount': -2532},
  {'transaction-id': 3225, 'amount': -2478},
  {'transaction-id': 3705, 'amount': -2368},
  {'transaction-id': 3725, 'amount': -2753},
  {'transaction-id': 3813, 'amount': -2698},
  {'transaction-id': 3817, 'amount': -2244},
  {'transaction-id': 4476, 'amount': -3024},
  {'transaction-id': 4561, 'amount': -2474},
  {'transaction-id': 4707, 'amount': -2091},
  {'transaction-id': 4847, 'amount': -2664},
  {'transaction-id': 4946, 'amount': -2426},
  {'transaction-id': 5356, 'amount': -2701},
  {'transaction-id': 5582, 'amount': -2186},
  {'transaction-id': 6020, 'amount': -2466},
  {'transaction

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 232, 'amount': -2199},
 {'transaction-id': 619, 'amount': -2681},
 {'transaction-id': 696, 'amount': -2537})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(-2199, -2681, -2537)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1157.1420122342763

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 167), ('Alice', 167), ('Alice', 168), ('Alice', 196), ('Bob', 105), ('Bob', 105), ('Bob', 106), ('Bob', 123), ('Charlie', 143), ('Charlie', 143), ('Charlie', 144), ('Charlie', 166), ('Dan', 72), ('Dan', 72), ('Dan', 72), ('Dan', 84), ('Edith', 96), ('Edith', 96), ('Edith', 96), ('Edith', 112), ('Frank', 94), ('Frank', 95), ('Frank', 96), ('Frank', 110), ('George', 84), ('George', 84), ('George', 84), ('George', 98), ('Hannah', 90), ('Hannah', 91), ('Hannah', 91), ('Hannah', 103), ('Ingrid', 60), ('Ingrid', 60), ('Ingrid', 60), ('Ingrid', 69), ('Jerry', 96), ('Jerry', 96), ('Jerry', 96), ('Jerry', 112), ('Kevin', 60), ('Kevin', 60), ('Kevin', 60), ('Kevin', 70), ('Laura', 102), ('Laura', 106), ('Laura', 106), ('Laura', 123), ('Michael', 168), ('Michael', 168), ('Michael', 168), ('Michael', 196), ('Norbert', 82), ('Norbert', 83), ('Norbert', 84), ('Norbert', 97), ('Oliver', 76), ('Oliver', 77), ('Oliver', 78), ('Oliver', 88), ('Patricia', 96), ('Patricia', 96), ('Patricia', 96

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 698), ('Bob', 439), ('Charlie', 596), ('Dan', 300), ('Edith', 400), ('Frank', 395), ('George', 350), ('Hannah', 375), ('Ingrid', 249), ('Jerry', 400), ('Kevin', 250), ('Laura', 437), ('Michael', 700), ('Norbert', 346), ('Oliver', 319), ('Patricia', 400), ('Quinn', 450), ('Ray', 697), ('Sarah', 899), ('Tim', 650), ('Ursula', 355), ('Victor', 420), ('Wendy', 348), ('Xavier', 550), ('Yvonne', 595), ('Zelda', 527)]
CPU times: user 114 ms, sys: 14.6 ms, total: 128 ms
Wall time: 495 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Charlie,"[{'transaction-id': 300, 'amount': -790}, {'tr..."
1,1,Quinn,"[{'transaction-id': 349, 'amount': 0}, {'trans..."
2,2,Edith,"[{'transaction-id': 578, 'amount': 56}, {'tran..."
3,3,Frank,"[{'transaction-id': 61, 'amount': 92}, {'trans..."
4,4,Patricia,"[{'transaction-id': 29, 'amount': 50}, {'trans..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 185 ms, sys: 6.51 ms, total: 192 ms
Wall time: 1.43 s


name
Alice      698
Bob        439
Charlie    596
Dan        300
Edith      400
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Charlie', 'amount': -790, 'transaction-id': 300},
 {'id': 0, 'name': 'Charlie', 'amount': -510, 'transaction-id': 331},
 {'id': 0, 'name': 'Charlie', 'amount': -658, 'transaction-id': 417})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Charlie,-790,300
1,0,Charlie,-510,331
2,0,Charlie,-658,417
3,0,Charlie,-665,654
4,0,Charlie,-682,838


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 160 ms, sys: 17.7 ms, total: 177 ms
Wall time: 1.11 s


name
Alice       29589
Bob         23876
Charlie     27344
Dan          6912
Edith       16409
Frank       11822
George      13101
Hannah      13073
Ingrid       6268
Jerry       30426
Kevin       12315
Laura       10104
Michael     38669
Norbert     12121
Oliver       6760
Patricia    23878
Quinn       22840
Ray         33826
Sarah       38694
Tim         21841
Ursula       6313
Victor       6821
Wendy       16058
Xavier      19759
Yvonne      25110
Zelda       26071
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()